# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
city_data = pd.read_csv('../WeatherPy/output_data/cities.csv')
city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mattru,7.6244,-11.8332,75.74,94,100,0.85,SL,1624178520
1,1,punta arenas,-53.1500,-70.9167,33.91,80,0,9.22,CL,1624178379
2,2,bluff,-46.6000,168.3333,39.79,89,63,11.97,NZ,1624178521
3,3,qasigiannguit,68.8193,-51.1922,46.65,71,97,7.09,GL,1624178236
4,4,ancud,-41.8697,-73.8203,45.23,87,99,5.86,CL,1624178522


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
coord = city_data[['Lat','Lng']]
humid = city_data['Humidity']
heat_layer = gmaps.heatmap_layer(coord, weights = humid, dissipating = False, max_intensity = 100, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
#Define weather conditions for the reduced dataframe

#Max Temperature Greater than 75 but less than 80 Farenheit

#Wind Speed less than 10 mph

#Cloudiness lower than 10 percent

In [30]:
vacation_city = city_data.loc[(city_data['Max Temp'] < 80) &
                              (city_data['Max Temp'] > 75) &
                              (city_data['Wind Speed'] < 10) & 
                              (city_data['Cloudiness'] < 10)]
vacation_city = vacation_city.reset_index(drop = True)
vacation_city

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,189,marsa matruh,31.3525,27.2453,79.00,65,0,9.22,EG,1624178617
1,291,port keats,-14.2500,129.5500,77.97,34,6,2.93,AU,1624178667
2,295,williamsburg,37.2707,-76.7075,79.09,80,1,5.75,US,1624178669
3,380,bafra,41.5678,35.9069,75.81,55,0,6.62,TR,1624178708
4,422,west wendover,40.7391,-114.0733,78.01,21,1,6.91,US,1624178727
5,436,siedlce,52.1677,22.2901,78.06,60,4,9.84,PL,1624178734


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
hotel_df = vacation_city
hotel_df['Hotel Name'] = ""
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,189,marsa matruh,31.3525,27.2453,79.00,65,0,9.22,EG,1624178617,
1,291,port keats,-14.2500,129.5500,77.97,34,6,2.93,AU,1624178667,
2,295,williamsburg,37.2707,-76.7075,79.09,80,1,5.75,US,1624178669,
3,380,bafra,41.5678,35.9069,75.81,55,0,6.62,TR,1624178708,
4,422,west wendover,40.7391,-114.0733,78.01,21,1,6.91,US,1624178727,


In [32]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius": 5000,
           "type": "lodging",
           "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    #get location from df
    params["location"] = f"{hotel_df['Lat'][index]},{hotel_df['Lng'][index]}"

    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"The first hotel within 5000m of {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"No hotel found in 5000 metres for {row['City']}")
        
    print("--------------------------------------------")


The first hotel within 5000m of marsa matruh is Beau Site.
--------------------------------------------
The first hotel within 5000m of port keats is TDC Village (Hundred Man Camp).
--------------------------------------------
The first hotel within 5000m of williamsburg is Comfort Suites Williamsburg Historic Area.
--------------------------------------------
The first hotel within 5000m of bafra is Sevgi Otel.
--------------------------------------------
The first hotel within 5000m of west wendover is Motel 6 Wendover, UT.
--------------------------------------------
The first hotel within 5000m of siedlce is Hotel Arche.
--------------------------------------------


In [33]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,189,marsa matruh,31.3525,27.2453,79.00,65,0,9.22,EG,1624178617,Beau Site
1,291,port keats,-14.2500,129.5500,77.97,34,6,2.93,AU,1624178667,TDC Village (Hundred Man Camp)
2,295,williamsburg,37.2707,-76.7075,79.09,80,1,5.75,US,1624178669,Comfort Suites Williamsburg Historic Area
3,380,bafra,41.5678,35.9069,75.81,55,0,6.62,TR,1624178708,Sevgi Otel
4,422,west wendover,40.7391,-114.0733,78.01,21,1,6.91,US,1624178727,"Motel 6 Wendover, UT"
5,436,siedlce,52.1677,22.2901,78.06,60,4,9.84,PL,1624178734,Hotel Arche


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(locations, info_box_content = hotel_info, fill_color = 'blue')
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))